In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [32]:
class temporal_network:
    
    #node-aligned
    #get binary edge pairs
    def __init__(self, size, length, edge_list, kind = 'temporal', omega = 1):
        
        self.nodes = [i for i in range(size)]
        self.time = length
        self.size = size
        adjacency = np.zeros((self.size*self.time,self.size*self.time))
        multi_array = np.zeros((self.time,self.size,self.size))
        
        #edge list is binary in the format of (i,j,t): 
        #there is an edge between i and j on time t(and an edge between j and i)
        for k,e in enumerate(edge_list):
            i,j,t = e[0],e[1],e[2]
            adjacency[self.size*(t-1)+i][self.size*(t-1)+j] = 1
            adjacency[self.size*(t-1)+j][self.size*(t-1)+i] = 1
            multi_array[t-1][i][j] = 1
            multi_array[t-1][j][i] = 1
            
        ##filling off diagonal blocks
        if kind == 'temporal':
            for n in range(self.size*(self.time-1)):
                adjacency[n][n+self.size] = omega
                adjacency[n+self.size][n] = omega
                
        elif kind == 'multilayer':
            i = 0
            while self.time-i != 0:
                i = i+1
                for n in range(self.size*(self.time-i)):
                    adjacency[n][n+i*self.size] = omega
                    adjacency[n+i*self.size][n] = omega
                    
        self.supra_adjacency = adjacency
        self.multi_array = multi_array
        
    def s_adjacency(self, multiarray, kind = 'temporal', omega=1):
        #function converting a multi dimensional array into supra adjacency matrix
        n = self.size
        t = self.time
        adjacency = np.zeros((self.size*self.time,self.size*self.time))
        
        for i,c in enumerate(multiarray[:]):
            adjacency[i*n:i*n+n,i*n:i*n+n] = c     
            
        if kind == 'temporal':
            for p in range(n*(t-1)):
                adjacency[p][p+n] = omega
                adjacency[p+n][p] = omega
                
        elif kind == 'multilayer':
            i = 0
            while t-i != 0:
                i = i+1
                for p in range(n*(t-i)):
                    adjacency[p][p+i*n] = omega
                    adjacency[p+i*n][p] = omega
        return(adjacency)
        
    def aggragate(self, normalized = True):
        t = self.time 
        n = self.size
        aggragated = np.zeros((n,n))
        for i,c in enumerate(self.multi_array[:]):
            aggragated = aggragated + c
        if normalized: return (aggragated/t)
        else: return (aggragated)
        
    def modularity_matrix(self, omega, gamma):
        N = self.size
        T = self.time
        B = np.zeros((N*T,N*T))
        two_mu = 0
        for i in range(T):
            k = np.sum(self.multi_array[i],0)
            two_m = np.sum(k,0)
            two_mu = two_mu + two_m
            B[i*N:(i+1)*N,i*N:(i+1)*N] = self.multi_array[i] - (gamma * k.T*k)/(two_m)
        two_mu = two_mu + 2*omega*N*(T-1)
        for p in range(N*(T-1)):
            B[p][p+N] = omega 
            B[p+N][p] = omega
        return(B)

In [30]:
edges = [[0,1,1],[0,1,2],[1,2,2],[0,1,3],[0,2,3],[0,1,4],[0,2,4],[1,2,4]]
T=temporal_network(3, 4, edges, kind = 'temporal', omega = 1)
S=temporal_network(3, 4, edges, kind = 'temporal', omega = 1).supra_adjacency
C=temporal_network(3, 4, edges, kind = 'temporal', omega = 1).multi_array
C

array([[[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 0.]],

       [[0., 1., 0.],
        [1., 0., 1.],
        [0., 1., 0.]],

       [[0., 1., 1.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[0., 1., 1.],
        [1., 0., 1.],
        [1., 1., 0.]]])

In [33]:
T.modularity_matrix(1,1)

array([[-0.5       ,  0.5       ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.5       , -0.5       ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-0.5       , -0.5       ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , -0.25      ,  0.        ,
        -0.25      ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.75      , -1.        ,
         0.75      ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        , -0.25      ,  0.        ,
        -0.25      ,  